<a target="_blank" href="https://colab.research.google.com/github/datafyresearcher/datafy-llm-workshop/blob/main/notebooks/04_LangChain_Agents.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Chains in LangChain

A LLM chain is the collection of steps in order to perform / follow instructions as per user input template. The chains enhances the cpability of LLMs and are the key building blocks of the generative-ai applicaions. As names suggested, the output of one component becomes the input of next component. Users can use the LLM model of their Choice. In this notebook we explore the OpenAI GPT-3.5.

## Most Common LLM Chains

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Conversational Retrieval QA (RAG Systems)
* Router Chain

[1] https://medium.com/dev-genius/llm-chains-theoretical-overview-5f8fdad3f081

In [3]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  # Install the package
  !pip install -q langchain==0.2.3 openai==1.34.0 litellm==1.40.9 langchain-community==0.2.4
else:
  print('Not running on CoLab')

Running on CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

#OpenAI API

Let us try first using Openai API directly

Read about the OpenAI API specifications and endpoints on below link

https://platform.openai.com/docs/api-reference

To learn about the available Models

https://platform.openai.com/docs/models/gpt-3-5

In [ ]:
import os
import openai

# select the model to use
llm_model = "gpt-3.5-turbo-0613"

# openai.api_key = ""

os.environ['OPENAI_API_KEY'] = ""

# os.environ["OPENAI_API_KEY"] = "sk-litellm-7_NPZhMGxY2GoHC59LgbDw" # [OPTIONAL] replace with your openai key

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [ ]:
#!pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/amjadraza/dlai-langchain/main/notebooks/langchain/Data.csv')

In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

Let us explore the very simple Chain implemented in LangChain Python Package.

In [4]:
from langchain.chat_models import ChatOpenAI, ChatLiteLLM
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
# Instantiate the LLM
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# Instantiate the liteLLM Model

llm_lite = ChatLiteLLM(model="gpt-3.5-turbo", temperature = 0.9)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best tagline to describe \
    a company that makes or produces or supply {product}?"
)

In [ ]:
chain = LLMChain(llm=llm_lite, prompt=prompt)

In [ ]:
product = "Generative AI"
response = chain.run(product)

In [ ]:
response

'"Unleashing the Potential of Generative AI"'

## SimpleSequentialChain

SImpleSequentialChain is another widely used LLMChain implemented in LangChain and serve as the key building block of AGIs.

The SimpleSequentialChain is a type of sequential chain in the Langchain library. It allows you to connect multiple chains and compose them into pipelines that execute a specific scenario. Each step in the chain has a singular input and output, where the output of one step is the input to the next.

In [5]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.7, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best tagline to describe \
    a company that makes or produces or supply {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a Business Proposal note more than 20 lines using \
   tagline :{tagline}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
response = overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Unleashing the Power of Generative AI for Limitless Possibilities"
Dear [Client's Name],

I hope this note finds you well. I am writing to present you with an exciting opportunity to harness the power of generative AI and unlock limitless possibilities for your business. Our tagline, "Unleashing the Power of Generative AI for Limitless Possibilities," perfectly encapsulates the potential that this technology holds for your organization.

At [Your Company Name], we specialize in developing cutting-edge AI solutions that drive innovation and help businesses thrive in the digital era. Generative AI, in particular, has emerged as a groundbreaking technology that can revolutionize various aspects of your operations, from product design to customer engagement.

By leveraging the power of generative AI, we can assist you in creating highly realistic and customizable virtual prototypes of your products. This will not only accelerate your product

## SequentialChain

In a SimpleSequentialChain, each step has a singular input and output, where the output of one step is the input to the next. This is useful when you want to take the output from one call and use it as the input to another.

In a SequentialChain, you can have multiple inputs and outputs. This is useful for more complex scenarios where you need to pass multiple variables between chains.

In [6]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )


In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [ ]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores, and the taste is much better...\nOld batch or counterfeit!?",
 'summary': "The reviewer finds the taste of the product mediocre, and suspects that it might be an old batch or counterfeit since the foam doesn't hold and the taste is better when bought from stores.",
 'followup_message': "Réponse de suivi:\n\nCher(e) critique,\n\nNous vous remercions d'avoir pris le temps de partager vos commentaires sur notre produit. Nous sommes désolés d'apprendre que la saveur ne vous a pas entièrement satisfait(e). Nous souhaitons souligner que notre produit est fabriqué avec des ingrédients de haute qualité et selon des normes rigoureuses de production. \n\nIl est possible que vous ayez reçu un lot plus ancien ou